In [1]:
import pandas as pd
import numpy as np
import os

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import inspect
from sqlalchemy.engine import reflection

In [2]:
df = pd.read_csv('data/Stations.csv')

In [3]:
df.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575


In [4]:
engine = create_engine("sqlite:///mta.db.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# session connection to db
session = Session(engine)

In [5]:
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

['stationLocationData']


### Read in Fares csv and Turnstiles Fares Sations csv to a Pandas DataFrame
#### Join both data frames on Remote column
#### Remove Duplicate columns
#### Send to Sqlite DB


In [10]:
dfOne = pd.read_csv('data/turnstiles_fares_stations2.csv')
dfTwo = pd.read_csv('data/fares_190209.csv')

In [29]:
# dfOne.head()

In [30]:
# dfTwo.head()

#### Merge Both tables on remote column


In [12]:
fareData = pd.merge(dfOne, dfTwo, on='REMOTE')

##### Set index to Station ID

In [14]:
fareData.set_index('Station ID', inplace=True)

In [41]:
# fareData.head(1)

##### Remove duplicate columns

In [23]:
fareData.drop(columns=[' STATION_x', ' STATION_y'], inplace=True)

In [28]:
# fareData.head(1)

#### Take updated fare data and add to sqllite DB

In [26]:
fareData.to_sql('fareData', con=engine)

In [27]:
# check to see that db has both tables
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

['fareData', 'stationLocationData']


### Read in turnstiles_190209.csv and turnstiles_fares_stations2.csv to a Pandas DataFrame
#### Join both data frames on Remote column
#### Remove Duplicate columns
#### Send to Sqlite DB


In [31]:
dfOne = pd.read_csv('data/turnstiles_fares_stations2.csv')
dfThree = pd.read_csv('data/turnstiles_190209.csv')

In [32]:
dfOne.head(1)

,UNIT,STATION,REMOTE,STATION,Station ID
0,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1


In [33]:
dfThree.head(1)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2/2/2019,3:00:00,REGULAR,6930882,2350417


In [35]:
turnstileData = pd.merge(dfOne, dfThree, on='UNIT')

In [37]:
turnstileData.head(1)

,UNIT,STATION_x,REMOTE,STATION,Station ID,C/A,SCP,STATION_y,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,3:00:00,REGULAR,10390146,12828793


In [39]:
turnstileData.drop(columns=['STATION_x', 'STATION_y'], inplace=True)

In [45]:
turnstileData.set_index('Station ID',inplace=True)
turnstileData.head(1)

,UNIT,REMOTE,STATION,C/A,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
Station ID,,,,,,,,,,,,
1,R095,R095,DITMARS BLVD & 31ST STREET,R515,00-03-00,NQW,BMT,2/2/2019,3:00:00,REGULAR,10390146,12828793


In [46]:
turnstileData.to_sql('turnStileData', con=engine)

ValueError: Table 'turnStileData' already exists.